# 1. Třídy (class)
Po prostorech jmen teď navážeme tématem objektů a tříd.
Třída je uživatelsky definovaný datový typ: popisuje data (atributy) i chování (metody), které mají její instance.

Definice se zapisuje klíčovým slovem `class` a odsazeným blokem.


In [ ]:
class MojeTrida:
    pass

## 1.1 Metody, atributy a konstruktor
Metoda je funkce definovaná uvnitř třídy.
Atribut je hodnota uložená v instanci (nebo na třídě).

Běžná instanční metoda má jako první parametr `self`, tedy konkrétní instanci.
Atributy instancí se obvykle nastavují v konstruktoru `__init__`.

Pokud třída `__init__` nedefinuje, použije se zděděná implementace (typicky z `object`).


In [ ]:
# ukázka jednoduché třídy MojeTrida
class MojeTrida:
    def __init__(self, parametr):
        self.parametr = parametr  # atribut

    def vypis_parametr(self):  # metoda
        print("Atribut má hodnotu:", self.parametr)


objekt = MojeTrida("Ahoj")  # vytvoření objektu pomocí konstruktoru
objekt.vypis_parametr()  # volání metody
print(objekt.parametr)  # přístup k atributu


## 1.2 Zapouzdření a `property`
Python technicky nevynucuje privátní atributy jako některé jiné jazyky.
Konvence je:

- `_atribut` znamená interní atribut (neměnit zvenku bez důvodu)
- `__atribut` spouští name mangling, ale není to bezpečnostní ochrana

`property` umožňuje řídit čtení a zápis atributu při zachování API `obj.hodnota`.
Hodí se třeba pro validaci nebo odvozené hodnoty.


In [ ]:
import math


class Kruh:
    def __init__(self, polomer):
        self.polomer = polomer

    @property
    def obsah(self):
        return math.pi * self.polomer ** 2

    @obsah.setter
    def obsah(self, hodnota_obsahu):
        print(f"Nastavuji obsah na {hodnota_obsahu}")
        self.polomer = math.sqrt(hodnota_obsahu / math.pi)


kruh = Kruh(1)
print(kruh.polomer)
print(kruh.obsah)

kruh.obsah = 3
print(kruh.polomer)
print(kruh.obsah)


## 1.3 Vybrané podtržítkové metody a atributy
V Pythonu mají podtržítkové prvky (`__...__`) pevně daný význam. Některé z nich:

- `__repr__` a `__str__`: textová reprezentace objektu
- `__getattr__` a `__setattr__`: práce s atributy
- `__call__`: volání objektu jako funkce
- `__doc__`, `__dict__`, `__class__`: dokumentace, jmenný prostor a třída objektu

Další pokrývají iteraci, operátory nebo chování kontejnerů.


In [ ]:
# ukázka __getattr__ a __setattr__
class MojeTrida:
    def __init__(self, parametr):
        self.parametr = parametr

    def __getattr__(self, jmeno_atributu):
        print("Přistupujete k neexistujícímu atributu:", jmeno_atributu)
        return "Ahoj"

    def __setattr__(self, jmeno_atributu, hodnota):
        print("Nastavuji atribut", jmeno_atributu, "na", hodnota)
        object.__setattr__(self, jmeno_atributu, hodnota)


objekt = MojeTrida("Ahoj")
print(dir(objekt))
print(objekt.tento_atribut_neexistuje)
objekt.novy_atribut = "Nazdarek"
print(objekt.novy_atribut)
print(dir(objekt))


In [ ]:
# ukázka __str__ a __repr__
# __str__ se používá pro čitelné zobrazení (např. print),
# __repr__ má vracet co nejjednoznačnější reprezentaci objektu.
class MojeTrida:
    def __init__(self, parametr):
        self.parametr = parametr

    def __str__(self):
        return "Toto je objekt třídy MojeTrida"

    def __repr__(self):
        return f"MojeTrida(parametr={self.parametr!r})"


objekt = MojeTrida("Ahoj")
print(objekt)
print(str(objekt))
print(repr(objekt))
objekt


### 1.3.1 `__format__` a f-string
Při formátování objektu ve f-stringu (`{obj:...}`) Python volá metodu `__format__`.


In [ ]:
class Bod3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __format__(self, format_spec):
        if format_spec == 'zyx':
            return f"{self.z}, {self.y}, {self.x}"
        elif format_spec == 'yzx':
            return f"{self.y}, {self.z}, {self.x}"
        else:
            return f"{self.x}, {self.y}, {self.z}"
        
bod = Bod3D(1, 2, 3)
print(f"{bod = }")
print(f"{bod:zyx}")
print(f"{bod:yzx}")
print(f"{bod = :}")


In [ ]:
# seznam atributů __dict__
class MojeTrida:
    def __init__(self, parametr1, parametr2):
        self.parametr1 = parametr1
        self.parametr2 = parametr2

    def vypis_parametry(self):
        print("Atributy mají hodnotu:", self.parametr1, ",", self.parametr2)


objekt = MojeTrida("Ahoj", "Sbohem")
print(objekt.__dict__)


In [ ]:
# ukázka __call__
class MojeTrida:
    def __init__(self, parametr):
        self.parametr = parametr

    def __call__(self, *args, **kwargs):
        print("Voláte objekt třídy MojeTrida")


objekt = MojeTrida("Ahoj")
objekt()


In [ ]:
# ukázka __doc__
class MojeTrida:
    """Ukázka dokumentace třídy MojeTrida."""

    def __init__(self, parametr):
        self.parametr = parametr

    def vypis_parametr(self):
        """Ukázka dokumentace metody vypis_parametr."""
        print("Atribut má hodnotu:", self.parametr)


objekt = MojeTrida("Ahoj")
print(objekt.__doc__)
print(objekt.vypis_parametr.__doc__)


### 1.3.2 `__getitem__` a `__setitem__`
Třída může implementovat `__getitem__` a `__setitem__`, takže s objektem pracujeme přes `[]` podobně jako se sekvencí nebo slovníkem.


In [ ]:
# ukázka __setitem__ a __getitem__
class MojeTrida:
    def __init__(self, prefix):
        self._data = {}
        self.prefix = prefix + " "

    def __setitem__(self, key, value):
        print("Nastavuji hodnotu atributu:", key, "na hodnotu:", value)
        self._data[key] = self.prefix + str(value)

    def __getitem__(self, item):
        return self._data[item]


objekt = MojeTrida("Ahoj")
objekt[0] = "Honzo"
objekt[1] = "Pepo"
objekt[2] = "Jardo"

print(objekt[0])
print(objekt[1])
print(objekt[2])


## 1.4 Operátory nad vlastními objekty
Chování operátorů můžeme definovat speciálními metodami, například:

- `__add__` pro `+`
- `__sub__` pro `-`
- `__mul__` pro `*`
- `__truediv__` pro `/`
- `__floordiv__` pro `//`
- `__mod__` pro `%`
- `__pow__` pro `**`
- `__and__`, `__or__`, `__xor__` pro bitové operátory

Kompletní seznam je v dokumentaci datového modelu Pythonu.


Metoda se nejdřív volá na levém operandu.
Když vrátí `NotImplemented` (nebo operace není podporovaná), Python zkusí odpovídající reverzní metodu na pravém operandu, např. `__radd__`.

Pokud neuspěje ani to, vyvolá se `TypeError`.


In [ ]:
class A:
    def __init__(self):
        pass

    def __add__(self, other):
        print("A.__add__")
        return self

    def __radd__(self, other):
        print("A.__radd__")
        return self


class B:
    def __init__(self):
        pass

    def __add__(self, other):
        print("B.__add__")
        return self


objekt_a = A()
objekt_b = B()

c = objekt_a + objekt_b
c = objekt_b + objekt_a
c = objekt_a + 5
c = 5 + objekt_a
c = 5 + objekt_b  # záměrně vyvolá TypeError: B nepodporuje + s int


### 1.4.1 Ukázka: komplexní čísla


In [ ]:
class KomplexniCislo:
    def __init__(self, realna_cast, imaginarni_cast):
        self.realna_cast = realna_cast
        self.imaginarni_cast = imaginarni_cast

    def __add__(self, other):
        return KomplexniCislo(self.realna_cast + other.realna_cast, self.imaginarni_cast + other.imaginarni_cast)

    def __sub__(self, other):
        return KomplexniCislo(self.realna_cast - other.realna_cast, self.imaginarni_cast - other.imaginarni_cast)

    def __mul__(self, other):
        realna = self.realna_cast * other.realna_cast - self.imaginarni_cast * other.imaginarni_cast
        imaginarni = self.imaginarni_cast * other.realna_cast + self.realna_cast * other.imaginarni_cast
        return KomplexniCislo(realna, imaginarni)

    def __truediv__(self, other):
        jmenovatel = other.realna_cast ** 2 + other.imaginarni_cast ** 2
        realna = (self.realna_cast * other.realna_cast + self.imaginarni_cast * other.imaginarni_cast) / jmenovatel
        imaginarni = (self.imaginarni_cast * other.realna_cast - self.realna_cast * other.imaginarni_cast) / jmenovatel
        return KomplexniCislo(realna, imaginarni)

    def __str__(self):
        znamenko = "+" if self.imaginarni_cast >= 0 else "-"
        return f"{self.realna_cast} {znamenko} {abs(self.imaginarni_cast)}i"

    def __repr__(self):
        return f"KomplexniCislo({self.realna_cast}, {self.imaginarni_cast})"


cislo1 = KomplexniCislo(1, 2)
cislo2 = KomplexniCislo(3, 4)

print(cislo1 + cislo2)
print(cislo1 - cislo2)
print(cislo1 * cislo2)
print(cislo1 / cislo2)
cislo1 / cislo2


## 1.5 Dědičnost
Dědičnost umožňuje převzít společné chování z rodičovské třídy a doplnit nebo přepsat jen rozdíly.

- dceřiná třída může přidat nové metody
- metody rodiče může přepsat (override)
- pokud dceřiná třída nedefinuje `__init__`, použije se zděděný konstruktor
- když vlastní `__init__` definuje a chce i inicializaci rodiče, musí ji zavolat explicitně (typicky `super().__init__(...)`)


In [ ]:
class Clovek:
    def __init__(self, jmeno):
        self.jmeno = jmeno

    def _rekni(self, text):
        print(self.jmeno, ":", text)

    def predstav_se(self):
        self._rekni("Jmenuji se " + self.jmeno + ".")

    def pozdrav(self):
        self._rekni("Dobrý den.")

    def rozluc_se(self):
        self._rekni("Nashledanou.")


class Elektrikar(Clovek):
    def oprav_televizi(self):  # nová metoda v dceřiné třídě
        self._rekni("Bude to v cuku letu.")
        print("--- Elektrikář něco šudlá. ---")
        self._rekni("A je to.")

    def predstav_se(self):  # přepsaná metoda rodičovské třídy
        self._rekni("Já jsem " + self.jmeno + ".")


class Zakaznik(Clovek):
    def nakupuj(self):  # nová metoda v dceřiné třídě
        self._rekni("Prosím, opravíte mi televizi?")


elektrikar = Elektrikar("Franta Vopička")
zakaznik = Zakaznik("Tomáš Marný")

# rozhovor
prubeh_rozhovoru = [
    zakaznik.pozdrav,
    elektrikar.pozdrav,
    zakaznik.predstav_se,
    elektrikar.predstav_se,
    zakaznik.nakupuj,
    elektrikar.oprav_televizi,
]
for akce in prubeh_rozhovoru:
    akce()


## 1.6 Dataclass
Dekorátor `@dataclass` zjednodušuje psaní tříd určených hlavně pro data.

Automaticky generuje například `__init__`, `__repr__` a `__eq__`.
Při `@dataclass(order=True)` se doplní i porovnávací metody (`__lt__`, `__le__`, `__gt__`, `__ge__`).


In [ ]:
from dataclasses import dataclass


@dataclass(order=True)
class Point3D:
    x: float
    y: float
    z: float


bod1 = Point3D(1, 2, 3)
bod2 = Point3D(1, 3, 2)

print(bod1)
print(bod1 < bod2)  # porovnání je lexikograficky podle pořadí polí


## 1.7 Další témata
Další oblasti, které je dobré znát, ale teď je nebudeme rozebírat:

- vícenásobná dědičnost
- metody třídy
- statické metody
- abstraktní třídy
- metatřídy
- návrhové vzory
